<a href="https://colab.research.google.com/github/BoyaShivatejaNaidu/ML-NLP/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gensim nltk

In [8]:
import nltk

# Downloads required for tokenization and stopwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk, shutil
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import nltk
import shutil
import os

# Remove broken nltk cache if it exists
shutil.rmtree('/root/nltk_data/tokenizers/punkt', ignore_errors=True)

# Clean redownload
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab') # Download the missing resource
print(word_tokenize("Colab NLTK is now working!"))

['Colab', 'NLTK', 'is', 'now', 'working', '!']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [7]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import nltk
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# NLTK Downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing resource

# Load your dataset
df = pd.read_csv("/content/spam.csv", encoding='ISO-8859-1')

# Rename and keep necessary columns (assuming structure matches standard spam.csv)
df = df[['v1', 'v2']]
df.columns = ['Label', 'Message']

# Preprocessing setup
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower(), language='english') # Explicitly specify language
    tokens = [t for t in tokens if t not in string.punctuation and t not in stop_words]
    return tokens

df['tokens'] = df['Message'].apply(preprocess)

# Load Google News Word2Vec (downloads ~1.5GB first time)
w2v = api.load("word2vec-google-news-300")

def vectorize(tokens, model):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(300)

df['vector'] = df['tokens'].apply(lambda x: vectorize(x, w2v))

# Prepare data
X = np.stack(df['vector'].values)
y = df['Label'].map({'ham': 0, 'spam': 1})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

# Predict function
def predict_message_class(model, w2v_model, message):
    tokens = preprocess(message)
    vec = vectorize(tokens, w2v_model).reshape(1, -1)
    pred = model.predict(vec)[0]
    return 'spam' if pred == 1 else 'ham'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[==================================================] 100.0% 1662.8/1662.8MB downloaded
Test Accuracy: 0.9443946188340807


In [8]:
def predict_message_class(model, w2v_model, message):
    tokens = preprocess(message)  # lowercase, tokenize, remove punctuation/stopwords
    vec = vectorize(tokens, w2v_model).reshape(1, -1)  # convert to vector
    pred = model.predict(vec)[0]
    return 'spam' if pred == 1 else 'ham'


In [11]:
msg = "Congrats! You've won a free vacation. Call now!"
print("Prediction:", predict_message_class(model, w2v, msg))  # → likely 'spam'


Prediction: ham
